In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def stats_metric(top, bottom, MAX_DIST=50):

    # big, small = None , None
    # if s1[3] > s2[3]:
    #     big = s1
    #     small = s2
    # else:
    #     big = s2
    #     small = s1
    # top, bottom = None, None
    # if s1[1] < s2[1]:
    #     top = s1
    #     bottom = s2
    # else:
    #     top = s2
    #     bottom = s1
    diff = abs(top[1] + top[3] - bottom[1])
    
    return diff < MAX_DIST

cap = cv2.VideoCapture ('vid1_IR.avi')
while (cap.isOpened()):
    ret, frame = cap.read()
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, bin = cv2.threshold(G, 45, 255, cv2.THRESH_BINARY)
    # bin = cv2.GaussianBlur(bin, (5, 5), 0)
    bin = cv2.medianBlur(bin, 3)
    bin = cv2.morphologyEx(bin, cv2.MORPH_CLOSE, np.ones((7, 7)))

    numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        bin)
    
    I_VIS = bin
    MAX_DIST=400
    RATIO = 1

    condition1 = stats[:, 4] > 200 
    condition2 = stats[:, 3] / stats[:, 2] > RATIO
    condition = condition1 & condition2
    stats = stats[condition, :]

    if (stats.shape[0] > 1): # are there any objects
        for s1 in range(1, stats.shape[0] - 1):
            for s2 in range(s1+1, stats.shape[0]):
                top, bottom = None, None
                if stats[s1, 1] < stats[s2, 1]:
                    top = stats[s1, :]
                    bottom = stats[s2, :]
                else:
                    top = stats[s2, :]
                    bottom = stats[s1, :]

                diff_y = bottom[1] - (top[1] + top[3])
                diff_x = abs(top[0] - bottom[0])
                if abs(diff_y) < MAX_DIST and diff_x < MAX_DIST:
                    top[3] += diff_y + bottom[3]
                    bottom[3] = 0   
                    # bottom[2] = 0

        con = stats[:, 3] > 0
        stats = stats[con, :]
    for pi in range(stats.shape[0]):
        cv2.rectangle(I_VIS, (stats[pi, 0], stats[pi, 1]), (stats[pi, 0] + stats[pi, 2], 
                    stats[pi , 1] + stats[pi, 3]), (255, 0, 0), 2)
        # cv2.putText(I_VIS, "%d" %pi, (int(centroids[pi, 0]), int(centroids[pi, 1])), 
        cv2.putText(I_VIS, "%d" %pi, (stats[pi, 0], stats[pi, 1]-5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
    
    # cv2.imshow('IR', bin)
    cv2.imshow('IR', I_VIS)
    if cv2.waitKey(1) & 0xFF == ord('q'): # break the loop when the ’q’ key is pressed
        break
cap.release ()